In [1]:
from doa_mts import *
from time import sleep
o1 = doaMtsOverlay('doa_mts.bit')

In [2]:
win_len = o1.control.gpio_control.axi_gpio_win_len.channel1[0:5]

In [3]:
dawf

NameError: name 'dawf' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants

win_len_power = 16 ## 0 to 25 power of window length
win_len.write(win_len_power)

fc = 5.7e9
min_ang = -90 # min scanning angle
max_ang = 90 # max scanning angle
scan_res = 1 # scan resolution
scan_axis = list(np.arange(min_ang, max_ang+1, scan_res)) # angles axis
num_elements = 4
# c = 299792458
lbd = scipy.constants.c / fc
d = lbd/2


channels = 8 # 4 I + 4 Q
N = len(o1.adc_capture) // channels

delay = 7

offset = 2**win_len_power
print(offset)

## Latency calculations
ram_depth = offset
freq = 125e6
clk_tick = 1/freq
ps_bw = 128
write_read = 2
raw_bw = 128
prod_bw = 512

latency_raw = write_read * ram_depth * clk_tick * raw_bw/ps_bw
latency_prod = write_read * ram_depth * clk_tick * prod_bw/ps_bw
print(latency_raw)
print(latency_prod)

In [ ]:
# while True:
## RAW
tic_raw = time.time() 
AlignedCaptureSamples = np.zeros((1,len(o1.adc_capture)),dtype=np.int16)
o1.internal_capture(AlignedCaptureSamples)

ShapedCaptureSamples = np.zeros((channels, N), dtype=np.int16)
for i in range(channels):
    ShapedCaptureSamples[i] = AlignedCaptureSamples[0][i::channels]

iqData = np.zeros((4, N), dtype=np.cdouble)
idx = 0
for i in range(0, channels, 2):
    iqData[idx] = np.conjugate(ShapedCaptureSamples[i] - 1j * ShapedCaptureSamples[i+1])
    idx = idx + 1
## RAW


rawData = iqData
rawData = rawData[:,delay:offset+delay]
R = rawData@np.conjugate(rawData.T)
print(len(rawData[0,:]))
# MVDR
Rx = R  # Data covariance matrix
Rx_Inv = np.linalg.inv(Rx)  # Inverse of covariance matrix
num_elements = 4  # Define the number of elements and set its value
A = np.zeros(num_elements, dtype=complex)

# Assuming scan_axis is already defined as a numpy array
yspec = np.zeros(len(scan_axis))

# Calculate yspec
for t in range(len(scan_axis)):
    A = np.exp(-1j * 2 * np.pi * d * np.arange(num_elements) * np.sin(np.radians(scan_axis[t])) / lbd)
    B = np.conj(np.transpose(A))@Rx_Inv@A
    yspec[t] = 10 * np.log10(np.abs(1 / B))

ang_raw = scan_axis[np.argmax(yspec)]
toc_raw = time.time()

power_old = round(np.max(yspec))
yspec_old = yspec

## IQ PROD
tic_prod = time.time() 

iq_prod = np.zeros((1,len(o1.adc_prod)),dtype=np.int32)
o1.prod_capture(iq_prod)

data = iq_prod[0]

mults = 16
length_data = len(data)
length_ = round(length_data / mults)
raw = np.zeros((mults, length_))

for i in range(mults):
    raw[i] = data[i::mults]

raw_s = raw
rawc = raw.astype(np.cdouble)

zer = np.zeros((1, rawc.shape[1]))
rawc = np.insert(rawc, [1, 8, 13, 16], [zer], axis=0)

rawData_prod = np.zeros(((mults + 4)//2, rawc.shape[1]), dtype=np.cdouble)
idd = 0
for n in range(0, mults + 4, 2):
    rawData_prod[idd, :] = np.conjugate(rawc[n, :] - 1j * rawc[n+1, :])
    idd += 1

# print(offset)
rawData_prod = rawData_prod[:, :offset]
print(len(rawData_prod[0,:]))

R[0,0] = np.mean(rawData_prod[0]) #0
R[0,1] = np.mean(rawData_prod[1])
R[0,2] = np.mean(rawData_prod[2])
R[0,3] = np.mean(rawData_prod[3])
R[1,0] = np.conj(R[0,1])          #4
R[1,1] = np.mean(rawData_prod[4])
R[1,2] = np.mean(rawData_prod[5])
R[1,3] = np.mean(rawData_prod[6])
R[2,0] = np.conj(R[0,2])          #8
R[2,1] = np.conj(R[1,2])
R[2,2] = np.mean(rawData_prod[7])
R[2,3] = np.mean(rawData_prod[8])
R[3,0] = np.conj(R[0,3])          #12
R[3,1] = np.conj(R[1,3])
R[3,2] = np.conj(R[2,3])
R[3,3] = np.mean(rawData_prod[9]) #15

# MVDR
Rx = R  # Data covariance matrix
Rx_Inv = np.linalg.inv(Rx)  # Inverse of covariance matrix
num_elements = 4  # Define the number of elements and set its value
A = np.zeros(num_elements, dtype=np.cdouble)

# Assuming scan_axis is already defined as a numpy array
yspec = np.zeros(len(scan_axis))

# Calculate yspec
for t in range(len(scan_axis)):
    A = np.exp(-1j * 2 * np.pi * d * np.arange(num_elements) * np.sin(np.radians(scan_axis[t])) / lbd)
    B = np.conj(np.transpose(A))@Rx_Inv@A
    yspec[t] = 10 * np.log10(np.abs(1 / B))

ang_prod = scan_axis[np.argmax(yspec)]
toc_prod = time.time() 
power = round(np.max(yspec))
koef = power_old-power


## fifo data
fifo_mat = np.zeros((1,len(o1.adc_fifo)),dtype=np.int32)
o1.fifo_capture(fifo_mat)
fifo_mat = fifo_mat[0,32:64]
tic_fifo = time.time() 

# fifo_mat = np.zeros((1,len(o1.adc_fifo)),dtype=np.int32)
# o1.fifo_capture(fifo_mat)
# fifo_mat = fifo_mat[0,32:64]
fpgaR = np.zeros((4, 4), dtype=np.cdouble)
idx = 0
for i in range(4):
    for j in range(4):
        fpgaR[i,j] = np.conjugate(fifo_mat[idx] - 1j * fifo_mat[idx+1])
        idx += 2

# MVDR
# R = fpgaR
Rx = fpgaR/10000  # Data covariance matrix
Rx_Inv = np.linalg.inv(Rx)  # Inverse of covariance matrix
num_elements = 4  # Define the number of elements and set its value
A = np.zeros(num_elements, dtype=np.cdouble)

# Assuming scan_axis is already defined as a numpy array
yspec_fifo = np.zeros(len(scan_axis))

# Calculate yspec
for t in range(len(scan_axis)):
    A = np.exp(-1j * 2 * np.pi * d * np.arange(num_elements) * np.sin(np.radians(scan_axis[t])) / lbd)
    B = np.conj(np.transpose(A))@Rx_Inv@A
    yspec_fifo[t] = 10 * np.log10(np.abs(1 / B))

ang_fifo = scan_axis[np.argmax(yspec_fifo)]    
toc_fifo = time.time() 

power_fifo = round(np.max(yspec_fifo))       
koef2 = power_fifo-power_old

# Plot data
fig = plt.figure()
# plt.subplot(211)
plt.plot(scan_axis, yspec_old, color='r', label='raw')
plt.plot(scan_axis, (yspec+koef), color='g', label='iq_prod')
plt.plot(scan_axis, (yspec_fifo-koef2), color='b', label='fifo') 
plt.xlabel('Scan Axis, degree')
plt.ylabel('Y-Spec, dB')
plt.title('MVDR DOA Spectrum from three sources')
plt.legend()
plt.grid(True)

print("Window length: {:.3f} samples".format(offset))
print("Resolution: {:.3f} deg".format(scan_res))
print("RAW: {:.3f} deg, IQ_prod: {:.3f} deg, FIFO: {:.3f} deg".format(ang_raw, ang_prod, ang_fifo))
t_raw = toc_raw - tic_raw
t_prod = toc_prod - tic_prod
t_fifo = toc_fifo - tic_fifo
print('{:.5f} sec RAW, {:.5f} sec IQ_prod, {:.5f} sec FIFO'.format(t_raw+latency_raw, t_prod+latency_prod, t_fifo))

# print("RAW: {:.3f} dB, IQ_prod: {:.3f} dB, FIFO: {:.3f} dB".format(power_old, power+koef, power_fifo-koef2))

plt.show()


In [ ]:
## Save DATA
from scipy.io import savemat
mdic1 = {"raw_d": iq_prod, "label": "raw_d"}
savemat("raw.mat", mdic1)
mdic2 = {"avg": fpgaR, "label": "avg"}
savemat("avg.mat", mdic2)


In [ ]:
A = np.zeros((len(scan_axis), 4), dtype=complex)
for t in range(len(scan_axis)):
    A[t, :] = np.exp(-1j * 2 * np.pi * d * np.arange(num_elements) * np.sin(np.radians(scan_axis[t])) / lbd)

In [ ]:
B = (np.matmul(A,Rx_Inv)*A.T)

In [ ]:
B = np.sum(A@Rx_Inv)@A.T

In [ ]:
# aaa = np.sum(np.matmul(A,Rx_Inv), 1)

# bbb = np.matmul(aaa, A.T)
# sss = np.sum(bbb)
aaa = (A@Rx_Inv)
bbb = np.sum(aaa, axis=0)
B =  bbb@np.conj(A.T)
np.sum((A@Rx_Inv), axis=0)@np.conj(A.T)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants
from scipy.io import savemat
from IPython.display import display, clear_output
from rfsoc4x2 import oled
oled = oled.oled_display()

win_len_power = 16 ## 0 to 25 power of window length
win_len.write(win_len_power)

fc = 5.7e9
min_ang = -90 # min scanning angle
max_ang = 90 # max scanning angle
scan_res = 0.01 # scan resolution
scan_axis = list(np.arange(min_ang, max_ang+1, scan_res)) # angles axis
num_elements = 4
lbd = scipy.constants.c / fc
d = lbd/2

A = np.zeros((len(scan_axis), 4), dtype=np.cdouble)
for t in range(len(scan_axis)):
    A[t, :] = np.exp(-1j * 2 * np.pi * d * np.arange(num_elements) * np.sin(np.radians(scan_axis[t])) / lbd)

channels = 8 # 4 I + 4 Q
N = len(o1.adc_capture) // channels

delay = 7

offset = 2**win_len_power
print(offset)

## Latency calculations
ram_depth = offset
freq = 125e6
clk_tick = 1/freq
ps_bw = 128
write_read = 2
raw_bw = 128
prod_bw = 512

latency_raw = write_read * ram_depth * clk_tick * raw_bw/ps_bw
latency_prod = write_read * ram_depth * clk_tick * prod_bw/ps_bw
print(latency_raw)
print(latency_prod)

while True:
    ## RAW
    tic_raw = time.time() 
    AlignedCaptureSamples = np.zeros((1,len(o1.adc_capture)),dtype=np.int16)
    o1.internal_capture(AlignedCaptureSamples)

    ShapedCaptureSamples = np.zeros((channels, N), dtype=np.int16)
    for i in range(channels):
        ShapedCaptureSamples[i] = AlignedCaptureSamples[0][i::channels]

    iqData = np.zeros((4, N), dtype=np.cdouble)
    idx = 0
    for i in range(0, channels, 2):
        iqData[idx] = np.conjugate(ShapedCaptureSamples[i] - 1j * ShapedCaptureSamples[i+1])
        idx = idx + 1
    ## RAW


    rawData = iqData
    rawData = rawData[:,delay:offset+delay]
    R = rawData@np.conjugate(rawData.T)
    print(len(rawData[0,:]))
    # MVDR
    Rx = R  # Data covariance matrix
    Rx_Inv = np.linalg.inv(Rx)  # Inverse of covariance matrix   

    # Assuming scan_axis is already defined as a numpy array
    yspec = np.zeros(len(scan_axis))

    # Calculate yspec
    # B = np.sum((A@Rx_Inv), axis=0)@np.conj(A.T)
    # B = np.zeros((len(scan_axis), 1), dtype=np.cdouble)
    B = np.sum((A@Rx_Inv), axis=0)@(A.T)
    yspec = 10 * np.log10(np.abs(1 / B))

    ang_raw = scan_axis[np.argmax(yspec)]
    toc_raw = time.time()

    power_old = (np.max(yspec))
    yspec_old = yspec

    ## fifo data
    fifo_mat = np.zeros((1,len(o1.adc_fifo)),dtype=np.int32)
    o1.fifo_capture(fifo_mat)
    fifo_mat = fifo_mat[0,32:64]
    tic_fifo = time.time() 

    # fifo_mat = np.zeros((1,len(o1.adc_fifo)),dtype=np.int32)
    # o1.fifo_capture(fifo_mat)
    # fifo_mat = fifo_mat[0,32:64]
    fpgaR = np.zeros((4, 4), dtype=np.cdouble)
    idx = 0
    for i in range(4):
        for j in range(4):
            fpgaR[i,j] = np.conjugate(fifo_mat[idx] - 1j * fifo_mat[idx+1])
            idx += 2

    # MVDR
    Rx = fpgaR  # Data covariance matrix
    Rx_Inv = np.linalg.inv(Rx)  # Inverse of covariance matrix

    # Assuming scan_axis is already defined as a numpy array
    yspec_fifo = np.zeros(len(scan_axis))

    # Calculate yspec
    # B = np.sum((A@Rx_Inv), axis=0)@np.conj(A.T)
    # B = np.zeros((len(scan_axis), 1), dtype=np.cdouble)
    B = np.sum((A@Rx_Inv), axis=0)@(A.T)
    yspec_fifo = 10 * np.log10(np.abs(1 / B))

    ang_fifo = scan_axis[np.argmax(yspec_fifo)]    
    toc_fifo = time.time() 

    power_fifo = (np.max(yspec_fifo))       
    koef2 = power_fifo-power_old

    # Plot data
    fig = plt.figure()
    plt.plot(scan_axis, yspec_old, color='r', label='raw')
    plt.plot(scan_axis, (yspec_fifo-koef2), color='b', label='fifo') 
    plt.xlabel('Scan Axis, degree')
    plt.ylabel('Y-Spec, dB')
    plt.title('MVDR DOA Spectrum from two sources')
    plt.legend()
    plt.grid(True)

    print("Window length: {:d} samples".format(offset))
    print("Resolution: {:.3f} deg".format(scan_res))
    print("RAW: {:.3f} deg, AVG: {:.3f} deg".format(ang_raw, ang_fifo))
    t_raw = toc_raw - tic_raw
    t_fifo = toc_fifo - tic_fifo
    print('{:.5f} sec RAW, {:.5f} sec AVG'.format(t_raw+latency_raw, t_fifo))
    oled.write("RAW: {:.3f} d \nAVG: {:.3f} d".format(ang_raw, ang_fifo))
    plt.show()
    clear_output(wait = True)

65529
Window length: 65536 samples
Resolution: 0.010 deg
RAW: 34.910 deg, AVG: 34.920 deg
0.08312 sec RAW, 0.01371 sec AVG


In [ ]:
    mdic1 = {"raw_d": iqData, "label": "raw_d"}
    savemat("raw.mat", mdic1)
    mdic2 = {"avg": fpgaR, "label": "avg", "win_len": 2**win_len_power, "label": "win_len"}
    savemat("avg.mat", mdic2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants
from scipy.io import savemat
from IPython.display import display, clear_output

win_len_power = 16 ## 0 to 25 power of window length
win_len.write(win_len_power)
iter_num = 100

iqData_m = []
fpgaR_m = []
for iter in range(iter_num):
    print('---')
    print(iter)
    ## RAW
    tic_raw = time.time() 
    AlignedCaptureSamples = np.zeros((1,len(o1.adc_capture)),dtype=np.int16)
    o1.internal_capture(AlignedCaptureSamples)

    ShapedCaptureSamples = np.zeros((channels, N), dtype=np.int16)
    for i in range(channels):
        ShapedCaptureSamples[i] = AlignedCaptureSamples[0][i::channels]

    iqData = np.zeros((4, N), dtype=np.cdouble)
    idx = 0
    for i in range(0, channels, 2):
        iqData[idx] = np.conjugate(ShapedCaptureSamples[i] - 1j * ShapedCaptureSamples[i+1])
        idx = idx + 1
        
    fifo_mat = np.zeros((1,len(o1.adc_fifo)),dtype=np.int32)
    o1.fifo_capture(fifo_mat)
    fifo_mat = fifo_mat[0,32:64]
    fpgaR = np.zeros((4, 4), dtype=np.cdouble)
    idx = 0
    for i in range(4):
        for j in range(4):
            fpgaR[i,j] = np.conjugate(fifo_mat[idx] - 1j * fifo_mat[idx+1])
            idx += 2
            
    iqData_m.append(iqData)
    fpgaR_m.append(fpgaR)
    clear_output(wait = True)

mdic1 = {"raw_d": iqData_m, "label": "raw_d"}
savemat("raw.mat", mdic1)
mdic2 = {"avg": fpgaR_m, "label": "avg", "win_len": 2**win_len_power, "label": "win_len"}
savemat("avg.mat", mdic2)
print('Done')